# Test net size



In [1]:
import tensorflow as tf
import LungQuant.nets.vnet_weights as vnet
import psutil
import humanize
import os

In [2]:
model = vnet.vnet((200,150,100,1))

2022-01-17 18:25:04.342947: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-17 18:25:04.343560: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 12. Tune using inter_op_parallelism_threads for best performance.


In [3]:
def surface_loss(true,pred):
    b_true = true[...,0]
    b_pred = pred[...,0]
    f_true = true[...,1]
    f_pred = pred[...,1]

    true_map = b_true - f_true
    multiplied = f_pred * true_map

    return tf.math.reduce_mean(multiplied)

def sdice(true, pred):
    b_true = true[...,0]
    b_pred = pred[...,0]
    f_true = true[...,1]
    f_pred = pred[...,1]

    true_map = f_true - b_true
    multiplied = f_pred * true_map

    return tf.math.reduce_mean(multiplied)

from tensorflow.keras import optimizers

adamlr = optimizers.Adam(
    lr=0.00001, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-08, 
    amsgrad=True)

In [4]:
model.compile(
        loss=surface_loss,
        optimizer=adamlr, 
        metrics=[sdice])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 150, 10 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 200, 150, 100 0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 200, 150, 100 0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 200, 150, 100 32          reshape_1[0][0]                  
______________________________________________________________________________________________

In [5]:
tf.config.experimental.get_memory_info("CPU")

AttributeError: module 'tensorflow_core._api.v2.config.experimental' has no attribute 'get_memory_info'

In [6]:
tf.config.list_physical_devices('CPU')

AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'list_physical_devices'

In [7]:
print(tf.__version__)

2.0.0


In [53]:
import numpy as np
x = np.random.rand(100,200,150,100,1)
y = np.zeros((100,200,150,100,2))
y[...,1] = np.ones((100,200,150,100))

x = tf.convert_to_tensor(x)
y = tf.convert_to_tensor(y)

dataset = tf.data.Dataset.from_tensor_slices((x,y))
dataset = dataset.batch(1)

In [54]:
#import GPUtil as GPU
#GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
printm()

Gen RAM Free: 8.8 GB  | Proc size: 3.9 GB


In [56]:
history=model.fit(
        dataset, 
        epochs=1000,
        verbose=1)

Epoch 1/1000
      1/Unknown - 6s 6s/step

TypeError: 'NoneType' object is not callable